In [1]:
import sys
sys.path.append('../../src/')

from lib.pysoarlib import *
import os

# Current working directory (.../tutorials)
cwd = os.path.abspath('')

# Planning & learning (chunking)

Here we'll look at writing programs that plan and learn.
Our program will plan with **look-ahead search** and learn with **chunking**.

### Planning 
> "In simple look-ahead planning, your program will try out operators internally to evaluate them before it commits to applying them in the actual task."

In the missionary problem, problem solving was internal so it didn't matter if you screwed up.
If you made a fatal mistake, you could just rerun the program, or undo your action.

In the real world, this often isn't plausible.
If the missionaries and cannibals problem affected real world people, we couldn't just undo a fatal action.
Instead, the agent will plan in its own head, so to speak, before taking an action.

### Learning 
After we do look-ahead search, we want our agent to *learn* from what it has found.

We'll use chunking to build rules that "summarize the comparisons and evaluations that occur in the look-ahead
search so that in the future the rules fire, making look-ahead search unnecessary - 
converting deliberation into reaction."

#### Chunking
> "[Chunking] is a form of explanation-based generalization. Within the examples in this section, chunking speeds up problem solving by creating search-control rules. In other types of problem solving, chunking creates other types of rules including elaborations, operator proposals and operator applications."

### Planning & Learning
> "Soar does not create a step-by-step sequence of operators to apply blindly, 
but instead learns a set of context-dependent rules that prefer or avoid specific operators 
for states in the space. As a result, 
Soar’s plans are more flexible so that plans learned for one problem may transfer to part of a second problem. 
It also makes it so that Soar only plans when it needs to."
 
 TODO ^^ Need an example of this
 
 ## The Water Jug Problem
 
Below we will import the same rules we wrote in the second tutorial.

In [3]:
from _5a_water_jug_rules import *

In [6]:
agent_raw = f"""
# Initialization
{initialize_water_jug}
# Goal state
{initialize_goal_state}
{detect_goal_state}
# Elaborations
{elaborate_empty_jug}
# Proposals
{propose_initialize_water_jug}
{propose_empty_indifferent}
{propose_pour_indifferent}
{propose_fill_indifferent}
# Application
{apply_empty}
{apply_fill}
{apply_pour__will_empty_jug}
{apply_pour__wont_empty_jug}
"""


jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

jug_agent.execute_command("step 10");

--------- SOURCING PRODUCTIONS ------------
Total: 12 productions sourced.
1:    O: O1 (initialize-water-jug)
2:    O: O3 (fill)
3:    O: O5 (empty)
4:    O: O6 (fill)
5:    O: O2 (fill)
6:    O: O8 (empty)
7:    O: O10 (pour)
8:    O: O16 (fill)
9:    O: O12 (empty)
10:    O: O18 (pour)


When we first wrote the water jug agent, we made it indifferent to selecting any operator. We did this because we *were not sure* what the best choice was. This is not the correct way to program agents. We should use the indifferent operator when we *are sure* that one operator is no better than another.

Let's undo that mistake and remove the indifferent preference from the `fill` operator.

In [8]:
propose_fill = """
sp {water-jug*propose*fill 
    (state <s> ^name water-jug
               ^jug <j>) 
    (<j> ^empty > 0)
-->
    (<s> ^operator <o> +) 
    (<o> ^name fill
         ^jug <j>)}
"""

# Removed "propose_fill_indifferent"
agent_raw = f"""
# Initialization
{initialize_water_jug}
# Goal state
{initialize_goal_state}
{detect_goal_state}
# Elaborations
{elaborate_empty_jug}
# Proposals
{propose_initialize_water_jug}
{propose_empty_indifferent}
{propose_pour_indifferent}
# Application
{apply_empty}
{apply_fill}
{apply_pour__will_empty_jug}
{apply_pour__wont_empty_jug}

# Added
{propose_fill}
"""

jug_agent = SoarAgent(
    agent_raw=agent_raw,
    config_filename=cwd + '/default.config',
)
jug_agent.add_connector('jug-agent', AgentConnector(jug_agent))
jug_agent.connect()

jug_agent.execute_command("step 10");

--------- SOURCING PRODUCTIONS ------------
Total: 12 productions sourced.
1:    O: O1 (initialize-water-jug)
2:    ==>S: S2 (operator tie)
3:       ==>S: S3 (state no-change)
4:          ==>S: S4 (state no-change)
5:             ==>S: S5 (state no-change)
6:                ==>S: S6 (state no-change)
7:                   ==>S: S7 (state no-change)
8:                      ==>S: S8 (state no-change)
9:                         ==>S: S9 (state no-change)
10:                            ==>S: S10 (state no-change)


When a Soar agent cannot reach a decision on what operator to select (during the decision procedure), it will enter a **substate**. 

What happened here was that Soar threw a **tie impasse** because the preferences were insufficient to pick a single operator.

If we print the augmentations on `s2`, we'll see that there are a few attributes/values on the state that we haven't seen before.


In [10]:
jug_agent.execute_command("p s2", print_res=True);

p s2
(S2 ^attribute operator ^choices multiple ^epmem E2 ^impasse tie ^item O2
       ^item O3 ^item-count 2 ^non-numeric O2 ^non-numeric O3
       ^non-numeric-count 2 ^quiescence t ^reward-link R4 ^smem L2
       ^superstate S1 ^type state)


`^impasse`, `tie` 

`^choices`, `multiple` 

`^item`, `O3`

`^item-count`, `2`

To handle a tie impasse, we want to determine which operator is the best to apply to the current state. 
We will achieve this by applying **evaluation** operators in the substate. 
An evaluation can be a number (likelihood of success) or a tag like "success", "failure", etc. This evaluation will be converted to a preference like "best" or "worst". 

The concept of resolving a tie impasse is called the **Selection** problem in Soar literature.
The selection problem is common across different domains.
It is so common that Soar provides a set of default utility rules for this problem.
We will be writing those ourselves in these tutorials, but visit p. 162 of the Soar Tutorial book to find out how to load them for future reference.

### Up next

1. **The state representation.** This includes the representation of evaluation objects that link evaluations with operators.
2. **The initial state creation rule.** Since this problem arises as a substate, an initial state is generated automatically.
3. **The operator proposal rules.** The only operator is evaluate-operator and it must be proposed for every tied operator.
4. **The operator application rules.** Evaluate-operator is implemented hierarchically, like the abstract operators in TankSoar. Therefore, operator application involves a substate and substate operators.
5. **The operator and state monitoring rules.** There are no special monitoring rules for Selection.
6. **The desired state recognition rule.** Desired state recognition is automatic. When there are sufficient
preferences, the decision procedure selects an operator, the impasse is resolved, and the substate is
removed from working memory.
7. **The failure recognition rule.** There are no failure states for this problem.
8. **The search control rules.** These rules help guide which operator should be evaluated first.


## 1. Selection State Representation

If the evaluation of a state will ultimately be a tag like "success" or "failure", they can be represented as an augmentation of the state (`^evaluation success`). We'll also have to include the operator that the evaluation refers to.

We will be adding the following to the state 

• `operator <o>` the identifier of the task operator being evaluated
• `symbolic-value success/partial-success/partial-failure/failure/indifferent`
• `numeric-value [number]`
• `value true` indicates that there is either a symbolic or numeric value.
• `desired <d>` the identifier of a desired state to use for evaluation if there is one


## 2. Selection Initial State Creation

As we saw above, `^type tie` is added to the state during an impasse. 
We can optionally add more information to the state, like the name `selection`.
"This rule really isn’t necessary, but will allow us to test for the name of the state instead of the impasse type in all of the remaining rules."

```
sp {default*selection*elaborate*name 
    :default
    (state <s> ^type tie) 
-->
    (<s> ^name selection)}
```

> This rule uses a new bit of syntax, the :default, which tells Soar that this is a default rule. Default rules do not behave any differently than other rules; however, Soar keeps separate statistics on default rules and allows you to remove all of the non-default rules easily.

## 3. Selection Operator Proposal

The selection problem only has one operator, the **evaluate-operator**. 
This operator looks for items that do not have an evaluation.
It will create an evaluation link and later computes the value.

```
Selection*propose*evaluate-operator
If the state is named selection and there is an item that does not have an evaluation with a value, 
then propose the evaluate-operator for that item.
```

In Soar, we'd write

```
sp {selection*propose*evaluate-operator 
    :default
    (state <s> ^name selection 
               ^item <i>)
    -{(state <s> ^evaluation <e>) 
      (<e>       ^operator   <i>
                 ^value      true)} 
-->
    (<s> ^operator <o> +, =) 
    (<o> ^name evaluate-operator
         ^operator <i>)}
```

Here we are testing that no evaluation exists on a state `<s>`.
We also want this operator to stay selected until an evaluation is created with a value.
This is why we also add the test `<e> ^value true`.

Only a single `evaluate-operator` will be created and applied for each of the tied operators.


## 4. Selection Operator Application

To apply the above rule, we need to create an evaluation data structure without a value.
We will also create additional structures in WM to make application simpler.
The end result will look like:

```
(<s> ^evaluation <e> 
     ^operator   <o>)
     
(<e> ^superoperator <so> 
     ^desired       <d>)
     
(<o> ^name               evaluate-operator 
     ^superoperator      <so>
     ^evaluation         <e> 
     ^superstate         <ss> 
     ^superproblem-space <sp>)
```

The desired structure `<d>` will be some data structure that matches the end goal of our task.
For the water jug problem, that goal is the three-gallon jug having a single gallon of water.

We will use the desired state to evaluate an operator. 
We can either look for a direct match to the desired state, or give a evaluation based off of how far off we are from the goal.

